# Notebook resposável por apresentar os componentes básicos do aggregate no mongo

ref.: https://www.practical-mongodb-aggregations.com/front-cover.html

In [1]:
from pymongo import MongoClient
from bson.son import SON
import pprint
import numpy as np
import pandas as pd

# para geração de dados fake
from faker import Faker
from pandasql import sqldf

## povoador

In [2]:
def criar_pessoa():
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    
    idade = np.random.randint(10, 30)
    
    pessoa = {
        'cpf': fake.cpf(),
        'nome': f'{primeiro_nome} {ultimo_nome}' ,
        'idade': idade ,
    }
    
    primeiro_nome = primeiro_nome.replace(" ", "")
    ultimo_nome = ultimo_nome.replace(" ", "")
    
    pessoa['email']= f'{primeiro_nome.lower()}.{ultimo_nome.lower()}@{fake.free_email_domain()}'
    
    return pessoa

## Configurações

In [3]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

In [4]:
client = MongoClient('localhost', 27017)
# Se a base de dados não existir, ela é criada automagicamente :-)
db = client['techshot_aggregate_basico']

In [5]:
lista_pessoas = []

for i in range(30):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

[{'cpf': '479.861.320-78',
  'email': 'theo.silveira@uol.com.br',
  'idade': 18,
  'nome': 'Theo Silveira'},
 {'cpf': '315.840.269-60',
  'email': 'mariana.sales@ig.com.br',
  'idade': 18,
  'nome': 'Mariana Sales'},
 {'cpf': '468.750.132-07',
  'email': 'rebeca.castro@bol.com.br',
  'idade': 11,
  'nome': 'Rebeca Castro'},
 {'cpf': '073.642.159-99',
  'email': 'davi.damata@ig.com.br',
  'idade': 27,
  'nome': 'Davi da Mata'},
 {'cpf': '589.241.307-14',
  'email': 'cecília.lopes@bol.com.br',
  'idade': 18,
  'nome': 'Cecília Lopes'},
 {'cpf': '862.579.034-83',
  'email': 'raquel.martins@gmail.com',
  'idade': 15,
  'nome': 'Raquel Martins'},
 {'cpf': '584.137.926-73',
  'email': 'isabelly.lopes@ig.com.br',
  'idade': 24,
  'nome': 'Isabelly Lopes'},
 {'cpf': '097.561.432-07',
  'email': 'joão.campos@uol.com.br',
  'idade': 20,
  'nome': 'João Campos'},
 {'cpf': '986.407.215-30',
  'email': 'guilherme.dacunha@ig.com.br',
  'idade': 10,
  'nome': 'Guilherme da Cunha'},
 {'cpf': '591.670.

In [6]:
df = pd.DataFrame.from_records(lista_pessoas)
df

,cpf,nome,idade,email
0,479.861.320-78,Theo Silveira,18,theo.silveira@uol.com.br
1,315.840.269-60,Mariana Sales,18,mariana.sales@ig.com.br
2,468.750.132-07,Rebeca Castro,11,rebeca.castro@bol.com.br
3,073.642.159-99,Davi da Mata,27,davi.damata@ig.com.br
4,589.241.307-14,Cecília Lopes,18,cecília.lopes@bol.com.br
5,862.579.034-83,Raquel Martins,15,raquel.martins@gmail.com
6,584.137.926-73,Isabelly Lopes,24,isabelly.lopes@ig.com.br
7,097.561.432-07,João Campos,20,joão.campos@uol.com.br
8,986.407.215-30,Guilherme da Cunha,10,guilherme.dacunha@ig.com.br
9,591.670.823-86,Maria Luiza Araújo,10,marialuiza.araújo@gmail.com


In [7]:
# se a coleção não existir, ela é criada automagicamente :-)
colecao_pessoa = db['pessoas']
pessoa_ids = colecao_pessoa.insert_many(lista_pessoas).inserted_ids
pessoa_ids

[ObjectId('62eec44504b27d5e6012c7b5'),
 ObjectId('62eec44504b27d5e6012c7b6'),
 ObjectId('62eec44504b27d5e6012c7b7'),
 ObjectId('62eec44504b27d5e6012c7b8'),
 ObjectId('62eec44504b27d5e6012c7b9'),
 ObjectId('62eec44504b27d5e6012c7ba'),
 ObjectId('62eec44504b27d5e6012c7bb'),
 ObjectId('62eec44504b27d5e6012c7bc'),
 ObjectId('62eec44504b27d5e6012c7bd'),
 ObjectId('62eec44504b27d5e6012c7be'),
 ObjectId('62eec44504b27d5e6012c7bf'),
 ObjectId('62eec44504b27d5e6012c7c0'),
 ObjectId('62eec44504b27d5e6012c7c1'),
 ObjectId('62eec44504b27d5e6012c7c2'),
 ObjectId('62eec44504b27d5e6012c7c3'),
 ObjectId('62eec44504b27d5e6012c7c4'),
 ObjectId('62eec44504b27d5e6012c7c5'),
 ObjectId('62eec44504b27d5e6012c7c6'),
 ObjectId('62eec44504b27d5e6012c7c7'),
 ObjectId('62eec44504b27d5e6012c7c8'),
 ObjectId('62eec44504b27d5e6012c7c9'),
 ObjectId('62eec44504b27d5e6012c7ca'),
 ObjectId('62eec44504b27d5e6012c7cb'),
 ObjectId('62eec44504b27d5e6012c7cc'),
 ObjectId('62eec44504b27d5e6012c7cd'),
 ObjectId('62eec44504b27d

### Pesquisa comum

Encontrar as pessoas que tenham mais de 18 anos

In [8]:
criterio = {
    'idade': {'$gt':18}
}
pp.pprint(criterio)

{'idade': {'$gt': 18}}


In [9]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

{'_id': ObjectId('62eec44504b27d5e6012c7b8'),
 'cpf': '073.642.159-99',
 'email': 'davi.damata@ig.com.br',
 'idade': 27,
 'nome': 'Davi da Mata'}


In [10]:
for p in colecao_pessoa.find(criterio):
    pp.pprint(p)

{'_id': ObjectId('62eec44504b27d5e6012c7b8'),
 'cpf': '073.642.159-99',
 'email': 'davi.damata@ig.com.br',
 'idade': 27,
 'nome': 'Davi da Mata'}
{'_id': ObjectId('62eec44504b27d5e6012c7bb'),
 'cpf': '584.137.926-73',
 'email': 'isabelly.lopes@ig.com.br',
 'idade': 24,
 'nome': 'Isabelly Lopes'}
{'_id': ObjectId('62eec44504b27d5e6012c7bc'),
 'cpf': '097.561.432-07',
 'email': 'joão.campos@uol.com.br',
 'idade': 20,
 'nome': 'João Campos'}
{'_id': ObjectId('62eec44504b27d5e6012c7c0'),
 'cpf': '762.943.108-22',
 'email': 'enzogabriel.cavalcanti@bol.com.br',
 'idade': 23,
 'nome': 'Enzo Gabriel Cavalcanti'}
{'_id': ObjectId('62eec44504b27d5e6012c7c1'),
 'cpf': '561.429.803-70',
 'email': 'igor.sales@hotmail.com',
 'idade': 21,
 'nome': 'Igor Sales'}
{'_id': ObjectId('62eec44504b27d5e6012c7c4'),
 'cpf': '871.563.092-77',
 'email': 'vitorgabriel.campos@yahoo.com.br',
 'idade': 23,
 'nome': 'Vitor Gabriel Campos'}
{'_id': ObjectId('62eec44504b27d5e6012c7c5'),
 'cpf': '243.175.896-64',
 'emai

## Mesma consulta utilizando aggregate

### Construção do pipeline

In [11]:
sql = """
SELECT *
FROM df
WHERE idade > 18
"""

print(sql)


SELECT *
FROM df
WHERE idade > 18



In [12]:
sqldf(sql, globals())

,cpf,nome,idade,email
0,073.642.159-99,Davi da Mata,27,davi.damata@ig.com.br
1,584.137.926-73,Isabelly Lopes,24,isabelly.lopes@ig.com.br
2,097.561.432-07,João Campos,20,joão.campos@uol.com.br
3,762.943.108-22,Enzo Gabriel Cavalcanti,23,enzogabriel.cavalcanti@bol.com.br
4,561.429.803-70,Igor Sales,21,igor.sales@hotmail.com
5,871.563.092-77,Vitor Gabriel Campos,23,vitorgabriel.campos@yahoo.com.br
6,243.175.896-64,Maria Eduarda Rodrigues,20,mariaeduarda.rodrigues@uol.com.br
7,956.487.321-55,Ana Laura Costa,24,analaura.costa@bol.com.br
8,169.237.548-28,João Vitor Dias,27,joãovitor.dias@yahoo.com.br
9,385.276.940-00,Luiz Gustavo Cardoso,24,luizgustavo.cardoso@ig.com.br


In [13]:
pipeline = [
    # apenas acima de 18 anos
    {"$match": {'idade': {'$gt': 18}}},
]

pipeline

[{'$match': {'idade': {'$gt': 18}}}]

In [14]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': ObjectId('62eec44504b27d5e6012c7b8'),
  'cpf': '073.642.159-99',
  'email': 'davi.damata@ig.com.br',
  'idade': 27,
  'nome': 'Davi da Mata'},
 {'_id': ObjectId('62eec44504b27d5e6012c7bb'),
  'cpf': '584.137.926-73',
  'email': 'isabelly.lopes@ig.com.br',
  'idade': 24,
  'nome': 'Isabelly Lopes'},
 {'_id': ObjectId('62eec44504b27d5e6012c7bc'),
  'cpf': '097.561.432-07',
  'email': 'joão.campos@uol.com.br',
  'idade': 20,
  'nome': 'João Campos'},
 {'_id': ObjectId('62eec44504b27d5e6012c7c0'),
  'cpf': '762.943.108-22',
  'email': 'enzogabriel.cavalcanti@bol.com.br',
  'idade': 23,
  'nome': 'Enzo Gabriel Cavalcanti'},
 {'_id': ObjectId('62eec44504b27d5e6012c7c1'),
  'cpf': '561.429.803-70',
  'email': 'igor.sales@hotmail.com',
  'idade': 21,
  'nome': 'Igor Sales'},
 {'_id': ObjectId('62eec44504b27d5e6012c7c4'),
  'cpf': '871.563.092-77',
  'email': 'vitorgabriel.campos@yahoo.com.br',
  'idade': 23,
  'nome': 'Vitor Gabriel Campos'},
 {'_id': ObjectId('62eec44504b27d5e6012c7c

### Consulta: quero exibir apenas o nome e a idade (independente da idade)

In [15]:
sql = """
SELECT nome, idade
FROM df
"""

print(sql)


SELECT nome, idade
FROM df



In [16]:
sqldf(sql, globals())

,nome,idade
0,Theo Silveira,18
1,Mariana Sales,18
2,Rebeca Castro,11
3,Davi da Mata,27
4,Cecília Lopes,18
5,Raquel Martins,15
6,Isabelly Lopes,24
7,João Campos,20
8,Guilherme da Cunha,10
9,Maria Luiza Araújo,10


In [17]:
pipeline = [
    # apenas os campos de interesse
    { "$project": { 
              "nome":1,
              "idade":1,
              "_id":0}
    },
]

pipeline

[{'$project': {'nome': 1, 'idade': 1, '_id': 0}}]

In [18]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'idade': 18, 'nome': 'Theo Silveira'}, {'idade': 18, 'nome': 'Mariana Sales'},
 {'idade': 11, 'nome': 'Rebeca Castro'}, {'idade': 27, 'nome': 'Davi da Mata'},
 {'idade': 18, 'nome': 'Cecília Lopes'},
 {'idade': 15, 'nome': 'Raquel Martins'},
 {'idade': 24, 'nome': 'Isabelly Lopes'}, {'idade': 20, 'nome': 'João Campos'},
 {'idade': 10, 'nome': 'Guilherme da Cunha'},
 {'idade': 10, 'nome': 'Maria Luiza Araújo'},
 {'idade': 16, 'nome': 'João Miguel Campos'},
 {'idade': 23, 'nome': 'Enzo Gabriel Cavalcanti'},
 {'idade': 21, 'nome': 'Igor Sales'}, {'idade': 13, 'nome': 'Carolina Ribeiro'},
 {'idade': 11, 'nome': 'Juan Oliveira'},
 {'idade': 23, 'nome': 'Vitor Gabriel Campos'},
 {'idade': 20, 'nome': 'Maria Eduarda Rodrigues'},
 {'idade': 12, 'nome': 'Enzo Gabriel Ramos'},
 {'idade': 12, 'nome': 'Augusto Fogaça'},
 {'idade': 24, 'nome': 'Ana Laura Costa'},
 {'idade': 27, 'nome': 'João Vitor Dias'},
 {'idade': 24, 'nome': 'Luiz Gustavo Cardoso'},
 {'idade': 12, 'nome': 'Arthur Dias'}, {'ida

### Consulta: quero exibir apenas o nome e a idade (apenas de quem possui acima de 18 anos)

In [19]:
sql = """
SELECT nome, idade
FROM df
WHERE idade > 18
"""

print(sql)


SELECT nome, idade
FROM df
WHERE idade > 18



In [20]:
sqldf(sql, globals())

,nome,idade
0,Davi da Mata,27
1,Isabelly Lopes,24
2,João Campos,20
3,Enzo Gabriel Cavalcanti,23
4,Igor Sales,21
5,Vitor Gabriel Campos,23
6,Maria Eduarda Rodrigues,20
7,Ana Laura Costa,24
8,João Vitor Dias,27
9,Luiz Gustavo Cardoso,24


In [21]:
pipeline = [
    # apenas acima de 18 anos
    {"$match": {'idade': {'$gt': 18}}},
    # apenas os campos de interesse
    { "$project": { 
              "nome":1,
              "idade":1,
              "_id":0}
    },    
]

pipeline

[{'$match': {'idade': {'$gt': 18}}},
 {'$project': {'nome': 1, 'idade': 1, '_id': 0}}]

In [22]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'idade': 27, 'nome': 'Davi da Mata'}, {'idade': 24, 'nome': 'Isabelly Lopes'},
 {'idade': 20, 'nome': 'João Campos'},
 {'idade': 23, 'nome': 'Enzo Gabriel Cavalcanti'},
 {'idade': 21, 'nome': 'Igor Sales'},
 {'idade': 23, 'nome': 'Vitor Gabriel Campos'},
 {'idade': 20, 'nome': 'Maria Eduarda Rodrigues'},
 {'idade': 24, 'nome': 'Ana Laura Costa'},
 {'idade': 27, 'nome': 'João Vitor Dias'},
 {'idade': 24, 'nome': 'Luiz Gustavo Cardoso'},
 {'idade': 28, 'nome': 'Pedro Lucas Fernandes'},
 {'idade': 21, 'nome': 'Sophie Moraes'},
 {'idade': 26, 'nome': 'Miguel da Costa'}]


In [23]:
# neste caso a ordem dos operadores não altera o resultado
pipeline = [
    # apenas os campos de interesse
    { "$project": { 
              "nome":1,
              "idade":1,
              "_id":0}
    },    
    # apenas aqueles acima de 18 anos
    {"$match": {'idade': {'$gt': 18}}},
]

pipeline

[{'$project': {'nome': 1, 'idade': 1, '_id': 0}},
 {'$match': {'idade': {'$gt': 18}}}]

In [24]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'idade': 27, 'nome': 'Davi da Mata'}, {'idade': 24, 'nome': 'Isabelly Lopes'},
 {'idade': 20, 'nome': 'João Campos'},
 {'idade': 23, 'nome': 'Enzo Gabriel Cavalcanti'},
 {'idade': 21, 'nome': 'Igor Sales'},
 {'idade': 23, 'nome': 'Vitor Gabriel Campos'},
 {'idade': 20, 'nome': 'Maria Eduarda Rodrigues'},
 {'idade': 24, 'nome': 'Ana Laura Costa'},
 {'idade': 27, 'nome': 'João Vitor Dias'},
 {'idade': 24, 'nome': 'Luiz Gustavo Cardoso'},
 {'idade': 28, 'nome': 'Pedro Lucas Fernandes'},
 {'idade': 21, 'nome': 'Sophie Moraes'},
 {'idade': 26, 'nome': 'Miguel da Costa'}]


### Consulta: 
- quero exibir apenas o nome e a idade 
- apenas de quem possui acima de 18 anos
- ordenado por idade

In [25]:
sql = """
SELECT nome, idade
FROM df
WHERE idade > 18
ORDER BY idade
"""

print(sql)


SELECT nome, idade
FROM df
WHERE idade > 18
ORDER BY idade



In [26]:
sqldf(sql, globals())

,nome,idade
0,João Campos,20
1,Maria Eduarda Rodrigues,20
2,Igor Sales,21
3,Sophie Moraes,21
4,Enzo Gabriel Cavalcanti,23
5,Vitor Gabriel Campos,23
6,Isabelly Lopes,24
7,Ana Laura Costa,24
8,Luiz Gustavo Cardoso,24
9,Miguel da Costa,26


In [27]:
pipeline = [
    # idade maior que 18
    {"$match": {'idade': {'$gt': 18}}},
    # apenas os campos de interesse
    { "$project": { 
              "nome":1,
              "idade":1,
              "_id":0}
    },
    # ordenando por idade
    # aqui uso SON pq python não mantem a ordem
     {'$sort': {'idade':1}},
    
]

pipeline

[{'$match': {'idade': {'$gt': 18}}},
 {'$project': {'nome': 1, 'idade': 1, '_id': 0}},
 {'$sort': {'idade': 1}}]

In [28]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'idade': 20, 'nome': 'João Campos'},
 {'idade': 20, 'nome': 'Maria Eduarda Rodrigues'},
 {'idade': 21, 'nome': 'Igor Sales'}, {'idade': 21, 'nome': 'Sophie Moraes'},
 {'idade': 23, 'nome': 'Enzo Gabriel Cavalcanti'},
 {'idade': 23, 'nome': 'Vitor Gabriel Campos'},
 {'idade': 24, 'nome': 'Isabelly Lopes'},
 {'idade': 24, 'nome': 'Ana Laura Costa'},
 {'idade': 24, 'nome': 'Luiz Gustavo Cardoso'},
 {'idade': 26, 'nome': 'Miguel da Costa'},
 {'idade': 27, 'nome': 'Davi da Mata'},
 {'idade': 27, 'nome': 'João Vitor Dias'},
 {'idade': 28, 'nome': 'Pedro Lucas Fernandes'}]


### Consulta: 
- quero exibir apenas o nome e a idade 
- apenas de quem possui acima de 18 anos
- ordenado primeiro por idade, segundo por nome

In [29]:
sql = """
SELECT nome, idade
FROM df
WHERE idade > 18
ORDER BY idade, nome
"""

print(sql)


SELECT nome, idade
FROM df
WHERE idade > 18
ORDER BY idade, nome



In [30]:
sqldf(sql, globals())

,nome,idade
0,João Campos,20
1,Maria Eduarda Rodrigues,20
2,Igor Sales,21
3,Sophie Moraes,21
4,Enzo Gabriel Cavalcanti,23
5,Vitor Gabriel Campos,23
6,Ana Laura Costa,24
7,Isabelly Lopes,24
8,Luiz Gustavo Cardoso,24
9,Miguel da Costa,26


In [31]:
df[df['idade'] > 18][['nome', 'idade']].sort_values(['idade'])

,nome,idade
7,João Campos,20
16,Maria Eduarda Rodrigues,20
12,Igor Sales,21
28,Sophie Moraes,21
11,Enzo Gabriel Cavalcanti,23
15,Vitor Gabriel Campos,23
6,Isabelly Lopes,24
19,Ana Laura Costa,24
21,Luiz Gustavo Cardoso,24
29,Miguel da Costa,26


In [32]:
pipeline = [
    # idade maior que 18
    {"$match": {'idade': {'$gt': 18}}},
    # apenas os campos de interesse
    { "$project": { 
              "nome":1,
              "idade":1,
              "_id":0}
    },
    # ordenando por idade
    # aqui uso SON pq python não mantem a ordem
     {'$sort': {'idade':1, 'nome':1}},
    
]

pipeline

[{'$match': {'idade': {'$gt': 18}}},
 {'$project': {'nome': 1, 'idade': 1, '_id': 0}},
 {'$sort': {'idade': 1, 'nome': 1}}]

In [33]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'idade': 20, 'nome': 'João Campos'},
 {'idade': 20, 'nome': 'Maria Eduarda Rodrigues'},
 {'idade': 21, 'nome': 'Igor Sales'}, {'idade': 21, 'nome': 'Sophie Moraes'},
 {'idade': 23, 'nome': 'Enzo Gabriel Cavalcanti'},
 {'idade': 23, 'nome': 'Vitor Gabriel Campos'},
 {'idade': 24, 'nome': 'Ana Laura Costa'},
 {'idade': 24, 'nome': 'Isabelly Lopes'},
 {'idade': 24, 'nome': 'Luiz Gustavo Cardoso'},
 {'idade': 26, 'nome': 'Miguel da Costa'},
 {'idade': 27, 'nome': 'Davi da Mata'},
 {'idade': 27, 'nome': 'João Vitor Dias'},
 {'idade': 28, 'nome': 'Pedro Lucas Fernandes'}]


### Consulta: 
- quero exibir apenas o nome e a idade 
- apenas de quem possui acima de 18 anos
- ordenado primeiro por idade (descrescente), segundo por nome

In [34]:
sql = """
SELECT nome, idade
FROM df
WHERE idade > 18
ORDER BY idade desc, nome
"""

print(sql)


SELECT nome, idade
FROM df
WHERE idade > 18
ORDER BY idade desc, nome



In [35]:
sqldf(sql, globals())

,nome,idade
0,Pedro Lucas Fernandes,28
1,Davi da Mata,27
2,João Vitor Dias,27
3,Miguel da Costa,26
4,Ana Laura Costa,24
5,Isabelly Lopes,24
6,Luiz Gustavo Cardoso,24
7,Enzo Gabriel Cavalcanti,23
8,Vitor Gabriel Campos,23
9,Igor Sales,21


In [36]:
pipeline = [
    # idade maior que 18
    {"$match": {'idade': {'$gt': 18}}},
    # apenas os campos de interesse
    { "$project": { 
              "nome":1,
              "idade":1,
              "_id":0}
    },
    # ordenando por idade
     {'$sort': {'idade':-1, 'nome':1}},
    
]

pipeline

[{'$match': {'idade': {'$gt': 18}}},
 {'$project': {'nome': 1, 'idade': 1, '_id': 0}},
 {'$sort': {'idade': -1, 'nome': 1}}]

In [37]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'idade': 28, 'nome': 'Pedro Lucas Fernandes'},
 {'idade': 27, 'nome': 'Davi da Mata'},
 {'idade': 27, 'nome': 'João Vitor Dias'},
 {'idade': 26, 'nome': 'Miguel da Costa'},
 {'idade': 24, 'nome': 'Ana Laura Costa'},
 {'idade': 24, 'nome': 'Isabelly Lopes'},
 {'idade': 24, 'nome': 'Luiz Gustavo Cardoso'},
 {'idade': 23, 'nome': 'Enzo Gabriel Cavalcanti'},
 {'idade': 23, 'nome': 'Vitor Gabriel Campos'},
 {'idade': 21, 'nome': 'Igor Sales'}, {'idade': 21, 'nome': 'Sophie Moraes'},
 {'idade': 20, 'nome': 'João Campos'},
 {'idade': 20, 'nome': 'Maria Eduarda Rodrigues'}]


### Consulta: 
- quero exibir apenas o nome e a idade 
- apenas de quem possui acima de 18 anos
- ordenado primeiro por idade (descrescente), segundo por nome
- limitando o resultado aos 5 primeiro

In [38]:
sql = """
SELECT nome, idade
FROM df
WHERE idade > 18
ORDER BY idade desc, nome
LIMIT 5
"""

print(sql)


SELECT nome, idade
FROM df
WHERE idade > 18
ORDER BY idade desc, nome
LIMIT 5



In [39]:
sqldf(sql, globals())

,nome,idade
0,Pedro Lucas Fernandes,28
1,Davi da Mata,27
2,João Vitor Dias,27
3,Miguel da Costa,26
4,Ana Laura Costa,24


In [40]:
pipeline = [
    # idade maior que 18
    {"$match": {'idade': {'$gt': 18}}},
    # apenas os campos de interesse
    { "$project": { 
              "nome":1,
              "idade":1,
              "_id":0}
    },
    # ordenando por idade
     {'$sort': {'idade':-1, 'nome':1}},
    # apenas os 5 primeiros resultados
    {'$limit':5}
]

pipeline

[{'$match': {'idade': {'$gt': 18}}},
 {'$project': {'nome': 1, 'idade': 1, '_id': 0}},
 {'$sort': {'idade': -1, 'nome': 1}},
 {'$limit': 5}]

In [41]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'idade': 28, 'nome': 'Pedro Lucas Fernandes'},
 {'idade': 27, 'nome': 'Davi da Mata'},
 {'idade': 27, 'nome': 'João Vitor Dias'},
 {'idade': 26, 'nome': 'Miguel da Costa'},
 {'idade': 24, 'nome': 'Ana Laura Costa'}]


## Funções de agregação (realizam algum cálculo sobre os documentos)

## ATENÇÃO: NESTE CASO A ORDEM DO PIPELINE IMPORTA!!!

### Consulta: 
- número de pessoas existentes na base


In [42]:
sql = """
SELECT count(*)
FROM df
"""

print(sql)


SELECT count(*)
FROM df



In [43]:
sqldf(sql, globals())

,count(*)
0,30


In [44]:
pipeline = [
    # agrupa para contar
    {
        '$group': {
            '_id': None,
            'count': { '$sum': 1 }
        }
    }]

pipeline

[{'$group': {'_id': None, 'count': {'$sum': 1}}}]

In [45]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': None, 'count': 30}]


### Consulta: 
- número de pessoas existentes na base acima de 18 anos


In [46]:
sql = """
SELECT count(*) as c
FROM df
WHERE idade > 18
"""

print(sql)


SELECT count(*) as c
FROM df
WHERE idade > 18



In [47]:
sqldf(sql, globals())

,c
0,13


In [48]:

pipeline = [
    # idade maior que 18
    {"$match": {'idade': {'$gt': 18}}},

    # agrupa para contar
    {
        '$group': {
            '_id': None,
            'c': { '$sum': 1 }
        }
    }]

pipeline

[{'$match': {'idade': {'$gt': 18}}},
 {'$group': {'_id': None, 'c': {'$sum': 1}}}]

In [49]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': None, 'c': 13}]


### Consulta: 
- somar a idade de todos participantes da base


In [50]:
sql = """
SELECT sum(idade) as total
FROM df
"""

print(sql)


SELECT sum(idade) as total
FROM df



In [51]:
sqldf(sql, globals())

,total
0,543


In [52]:
pipeline = [
    # agrupa para contar
    {
        '$group': {
            '_id': None,
            'total': { '$sum': '$idade' }
        }
    }]

pipeline

[{'$group': {'_id': None, 'total': {'$sum': '$idade'}}}]

In [53]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': None, 'total': 543}]


### Consulta: 
- para todos os participantes da base:
- somar a idade
- maior idade
- menor idade
- média de idades


In [54]:
sql = """
SELECT sum(idade) as total,
max(idade) ma,
min(idade) mi,
avg(idade) me
FROM df
"""

print(sql)


SELECT sum(idade) as total,
max(idade) ma,
min(idade) mi,
avg(idade) me
FROM df



In [55]:
sqldf(sql, globals())

,total,ma,mi,me
0,543,28,10,18.1


In [56]:
pipeline = [
    # agrupa para realizar os cálculos
    {
        '$group': {
            '_id': None,
            'total': { '$sum': '$idade' },
            'ma': { '$max': '$idade' },
            'mi': { '$min': '$idade' },
            'me': { '$avg': '$idade' },
        }
    }]

pipeline

[{'$group': {'_id': None,
   'total': {'$sum': '$idade'},
   'ma': {'$max': '$idade'},
   'mi': {'$min': '$idade'},
   'me': {'$avg': '$idade'}}}]

In [57]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': None, 'ma': 28, 'me': 18.1, 'mi': 10, 'total': 543}]


### Consulta: 
- somar a idade de todos participantes da base (apenas daqueles que possuem acima de 18 anos)


In [58]:
sql = """
SELECT sum(idade) as total
FROM df
WHERE idade > 18
"""

print(sql)


SELECT sum(idade) as total
FROM df
WHERE idade > 18



In [59]:
sqldf(sql, globals())

,total
0,308


In [60]:
pipeline = [
    # idade maior que 18
    {"$match": {'idade': {'$gt': 18}}},

    # agrupa para contar
    {
        '$group': {
            '_id': None,
            'total': { '$sum': '$idade' }
        }
    }]

pipeline

[{'$match': {'idade': {'$gt': 18}}},
 {'$group': {'_id': None, 'total': {'$sum': '$idade'}}}]

In [61]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': None, 'total': 308}]


### Consulta: 
- contar quantas pessoas existem por idade


In [62]:
sql = """
SELECT idade, count(*) as c
FROM df
GROUP BY idade
"""

print(sql)


SELECT idade, count(*) as c
FROM df
GROUP BY idade



In [63]:
sqldf(sql, globals())

,idade,c
0,10,2
1,11,2
2,12,4
3,13,1
4,14,1
5,15,1
6,16,2
7,17,1
8,18,3
9,20,2


In [64]:
pipeline = [
    # agrupa para contar
    {
        '$group': {
            '_id': "$idade",
            'c': { '$sum': 1 }
        }
    }]

pipeline

[{'$group': {'_id': '$idade', 'c': {'$sum': 1}}}]

In [65]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': 10, 'c': 2}, {'_id': 13, 'c': 1}, {'_id': 26, 'c': 1},
 {'_id': 17, 'c': 1}, {'_id': 15, 'c': 1}, {'_id': 14, 'c': 1},
 {'_id': 24, 'c': 3}, {'_id': 23, 'c': 2}, {'_id': 28, 'c': 1},
 {'_id': 20, 'c': 2}, {'_id': 16, 'c': 2}, {'_id': 18, 'c': 3},
 {'_id': 27, 'c': 2}, {'_id': 21, 'c': 2}, {'_id': 12, 'c': 4},
 {'_id': 11, 'c': 2}]


### Consulta: 
- contar quantas pessoas existem por idade, apenas aquelas pessoas com idade repetida


In [66]:
sql = """
SELECT idade, count(*) as c
FROM df
GROUP BY idade
HAVING c>1
"""

print(sql)


SELECT idade, count(*) as c
FROM df
GROUP BY idade
HAVING c>1



In [67]:
sqldf(sql, globals())

,idade,c
0,10,2
1,11,2
2,12,4
3,16,2
4,18,3
5,20,2
6,21,2
7,23,2
8,24,3
9,27,2


In [68]:
pipeline = [
    # agrupa para contar
    {
        '$group': {
            '_id': "$idade",
            'c': { '$sum': 1 }
        }
    },
    # recupera apenas aqueles cuja contagem seja > 1
    { '$match': { 'c': { '$gt': 1 } } }
]

pipeline

[{'$group': {'_id': '$idade', 'c': {'$sum': 1}}},
 {'$match': {'c': {'$gt': 1}}}]

In [69]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': 16, 'c': 2}, {'_id': 24, 'c': 3}, {'_id': 23, 'c': 2},
 {'_id': 20, 'c': 2}, {'_id': 18, 'c': 3}, {'_id': 21, 'c': 2},
 {'_id': 27, 'c': 2}, {'_id': 12, 'c': 4}, {'_id': 10, 'c': 2},
 {'_id': 11, 'c': 2}]


Para mais operadores de funções agregadas: https://www.mongodb.com/docs/manual/reference/operator/aggregation/